## Introduction

The data for this project is pulled from the Boston Blue Bikes [website](https://www.bluebikes.com/system-data). The data is being pulled from the historical data page, where there are shown to be several file naming conventions over the past decade. This corresponds to changes in the physical infrastructure (expansion of bike stations, etc) and changes in the organizational structures (Hubway to Boston Blue Bikes). 

We need to verify if these naming conventions also apply to changes in the file structures, so that we can adjust the dataframes before uploading to GCP.

## File types

There are two main subsets of data files available in this directory: trip data and station data. 

### Trip data

The trip data files follow two naming conventions:
- YYYYMM-hubway-tripdata.zip
- YYYYMM-bluebikes-tripdata.zip

Among these two naming conventions, there are three sets of data fields that need to be reconciled. In this project, I will only be including the fields which are present in all three sets. In addition, I will be dropping rows where any value in the row is null or another ineligible value.

## Trip Data

### hubway_Trips_YYYY.zip

These compressed files contain a single csv file, and so that can be opened with the pandas.read_csv() method. 

In [46]:
import pandas as pd
import numpy as np

df = pd.read_csv("https://s3.amazonaws.com/hubway-data/hubway_Trips_2011.csv")
df.head(10)

/tmp/ipykernel_660821/327739768.py:4: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("https://s3.amazonaws.com/hubway-data/hubway_Trips_2011.csv")


,Duration,Start date,End date,Start station number,Start station name,End station number,End station name,Bike number,Member type,Zip code,Gender
0,1712320,11/30/2011 23:58,12/1/2011 0:26,D32005,Boston Public Library - 700 Boylston St.,D32011,Stuart St. at Charles St.,B00056,Member,2116,Male
1,313200,11/30/2011 23:56,12/1/2011 0:01,C32008,Boylston at Fairfield,D32011,Stuart St. at Charles St.,B00133,Casual,NaN,NaN
2,1111430,11/30/2011 23:18,11/30/2011 23:36,A32009,Tremont St / W Newton St,D32006,Lewis Wharf - Atlantic Ave.,B00471,Member,2109,Male
3,1313487,11/30/2011 23:15,11/30/2011 23:37,A32001,Union Square - Brighton Ave. at Cambridge St.,D32005,Boston Public Library - 700 Boylston St.,B00056,Member,2116,Male
4,345115,11/30/2011 22:59,11/30/2011 23:05,B32008,Mayor Martin J. Walsh - 28 State St.,D32006,Lewis Wharf - Atlantic Ave.,B00174,Member,2109,Male
5,904843,11/30/2011 22:48,11/30/2011 23:03,A32004,Longwood Ave / Binney St,A32009,Tremont St / W Newton St,B00203,Member,2118,Male
6,266305,11/30/2011 22:42,11/30/2011 22:47,D32005,Boston Public Library - 700 Boylston St.,C32000,Tremont St. at Berkeley St.,B00431,Member,2118,Male
7,2065156,11/30/2011 22:40,11/30/2011 23:14,D32005,Boston Public Library - 700 Boylston St.,D32014,Tremont St / West St,B00028,Member,2114,Female
8,2093619,11/30/2011 22:39,11/30/2011 23:14,D32005,Boston Public Library - 700 Boylston St.,D32014,Tremont St / West St,B00394,Member,2111,Male
9,210672,11/30/2011 21:57,11/30/2011 22:00,D32011,Stuart St. at Charles St.,C32000,Tremont St. at Berkeley St.,B00381,Member,2116,Female


In [47]:
def clean_tripdata_old(df: pd.DataFrame) -> pd.DataFrame:
    """All columns need to be renamed and retyped to match newer trips datasets"""

    # tripduration
    df["tripduration"] = df["Duration"]
    df["tripduration"] = pd.to_numeric(df["tripduration"], downcast="integer")
    df = df.drop(["Duration"], axis=1)
    df = df[df["tripduration"] < 4.32e5]

    # starttime
    df["starttime"] = df["Start date"]
    df["starttime"] = pd.to_datetime(df["starttime"])
    df = df.drop(["Start date"], axis=1)

    # stoptime
    df["stoptime"] = df["End date"]
    df["stoptime"] = pd.to_datetime(df["stoptime"])
    df = df.drop(["End date"], axis=1)

    # start station id
    df["start station id"] = df["Start station number"]
    df = df.drop(["Start station number"], axis=1)

    # start station name
    df["start station name"] = df["Start station name"].str.lower()
    df = df.drop(["Start station name"], axis=1)

    # start station latitude
    df["start station latitude"] = np.nan

    # start station longitude
    df["start station longitude"] = np.nan

    # end station id
    df["end station id"] = df["End station number"]
    df = df.drop(["End station number"], axis=1)

    # end station name
    df["end station name"] = df["End station name"].str.lower()
    df = df.drop(["End station name"], axis=1)

    # end station latitude
    df["end station latitude"] = np.nan

    # end station longitude
    df["end station longitude"] = np.nan

    # bikeid
    df["bikeid"] = df["Bike number"]
    df = df.drop(["Bike number"], axis=1)

    # usertype
    df["usertype"] = df["Member type"].str.lower()
    df = df.drop(["Member type"], axis=1)

    # Zip code

    df["postal code"] = df["Zip code"].astype(str)
    df = df.drop(["Zip code"], axis=1)

    df["postal code"] = pd.to_numeric(
        df["postal code"], downcast="integer", errors="coerce"
    )

    # birth year
    df["birth year"] = np.nan

    # gender
    df["gender"] = df["Gender"]
    df.loc[(df["gender"] != "Male") & (df["gender"] != "Female"), "gender"] = 0
    df.loc[df["gender"] == "Male", "gender"] = 1
    df.loc[df["gender"] == "Female", "gender"] = 2
    df["gender"] = pd.to_numeric(df["gender"], downcast="float")
    df = df.drop(["Gender"], axis=1)

    return df

cleaned_df = clean_tripdata_old(df)
cleaned_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,postal code,birth year,gender
1,313200,2011-11-30 23:56:00,2011-12-01 00:01:00,C32008,boylston at fairfield,NaN,NaN,D32011,stuart st. at charles st.,NaN,NaN,B00133,casual,NaN,NaN,0.0
4,345115,2011-11-30 22:59:00,2011-11-30 23:05:00,B32008,mayor martin j. walsh - 28 state st.,NaN,NaN,D32006,lewis wharf - atlantic ave.,NaN,NaN,B00174,member,2109.0,NaN,1.0
6,266305,2011-11-30 22:42:00,2011-11-30 22:47:00,D32005,boston public library - 700 boylston st.,NaN,NaN,C32000,tremont st. at berkeley st.,NaN,NaN,B00431,member,2118.0,NaN,1.0
9,210672,2011-11-30 21:57:00,2011-11-30 22:00:00,D32011,stuart st. at charles st.,NaN,NaN,C32000,tremont st. at berkeley st.,NaN,NaN,B00381,member,2116.0,NaN,2.0
11,410857,2011-11-30 21:41:00,2011-11-30 21:47:00,A32010,south station - 700 atlantic ave.,NaN,NaN,D32017,the esplanade - beacon st. at arlington st.,NaN,NaN,B00241,member,2116.0,NaN,1.0


In [48]:
print(cleaned_df.dtypes)
print("\n")
print(cleaned_df.shape)
print("\n")
cleaned_df.describe()

tripduration                        int32
starttime                  datetime64[ns]
stoptime                   datetime64[ns]
start station id                   object
start station name                 object
start station latitude            float64
start station longitude           float64
end station id                     object
end station name                   object
end station latitude              float64
end station longitude             float64
bikeid                             object
usertype                           object
postal code                       float64
birth year                        float64
gender                            float32
dtype: object


(35272, 16)




,tripduration,start station latitude,start station longitude,end station latitude,end station longitude,postal code,birth year,gender
count,35272.000000,0.0,0.0,0.0,0.0,29552.000000,0.0,35272.000000
mean,300279.423169,NaN,NaN,NaN,NaN,2260.768848,NaN,0.979190
std,85441.388165,NaN,NaN,NaN,NaN,2473.133709,NaN,0.550547
min,60021.000000,NaN,NaN,NaN,NaN,216.000000,NaN,0.000000
25%,237282.500000,NaN,NaN,NaN,NaN,2109.000000,NaN,1.000000
50%,309973.000000,NaN,NaN,NaN,NaN,2118.000000,NaN,1.000000
75%,372441.000000,NaN,NaN,NaN,NaN,2176.000000,NaN,1.000000
max,431999.000000,NaN,NaN,NaN,NaN,84010.000000,NaN,2.000000


### Files named YYYYMM-hubway-tripdata.zip

These compressed files contain a single csv file, and so that can be opened with the pandas.read_csv() method. 

In [107]:
import pandas as pd

df_hubway = pd.read_csv("https://s3.amazonaws.com/hubway-data/201503-hubway-tripdata.zip")
df_hubway = df_hubway.drop(columns=['birth year', 'gender'])
df_hubway.dtypes

tripduration                 int64
starttime                   object
stoptime                    object
start station id             int64
start station name          object
start station latitude     float64
start station longitude    float64
end station id              object
end station name            object
end station latitude        object
end station longitude       object
bikeid                       int64
usertype                    object
dtype: object

In [106]:
df_hubway.bikeid.value_counts()

1098    72
883     64
1302    61
660     61
1086    60
        ..
749      7
869      7
1022     6
665      5
1288     3
Name: bikeid, Length: 321, dtype: int64

In [108]:
def clean_tripdata(df: pd.DataFrame) -> pd.DataFrame:
    """Columns retyped to increase performance"""

    # remove non-overlapping columns
    if "birth year" in df.columns:
        df = df.drop(["birth year"], axis=1)

    if "gender" in df.columns:
        df = df.drop(["gender"], axis=1)

    if "postal code" in df.columns:
        df = df.drop(["postal code"], axis=1)

    # tripduration
    df.tripduration = df.tripduration.astype("int32")
    df = df[df.tripduration.values < 4.32e5]

    # starttime
    df.starttime = pd.to_datetime(df.starttime)

    # stoptime
    df.stoptime = pd.to_datetime(df.stoptime)

    # start_station_id
    df.loc[:, "start_station_id"] = df.loc[:, "start station id"].astype("int16")
    df = df.drop(["start station id"], axis=1)

    # start_station_name
    df.loc[:, "start_station_name"] = df.loc[:, "start station name"].astype("string")
    df.start_station_name = df.start_station_name.str.lower()
    df = df.drop(["start station name"], axis=1)

    # start_station_latitude
    df.loc[:, "start_station_latitude"] = df.loc[:, "start station latitude"].astype(
        "float64"
    )
    df = df.drop(["start station latitude"], axis=1)

    # start_station_longitude
    df.loc[:, "start_station_longitude"] = df.loc[:, "start station longitude"].astype(
        "float64"
    )
    df = df.drop(["start station longitude"], axis=1)

    # end_station_id
    df = df[df.loc[:, "end station id"] != r"\N"]
    df.loc[:, "end_station_id"] = df.loc[:, "end station id"].astype("int16")
    df = df.drop(["end station id"], axis=1)

    # end_station_name
    df.loc[:, "end_station_name"] = df.loc[:, "end station name"].astype("string")
    df.end_station_name = df.end_station_name.str.lower()
    df = df.drop(["end station name"], axis=1)

    # end_station_latitude
    df = df[df.loc[:, "end station latitude"] != r"\N"]
    df.loc[:, "end_station_latitude"] = df.loc[:, "end station latitude"].astype(
        "float64"
    )
    df = df.drop(["end station latitude"], axis=1)

    # end_station_longitude
    df = df[df.loc[:, "end station longitude"] != r"\N"]
    df.loc[:, "end_station_longitude"] = df.loc[:, "end station longitude"].astype(
        "float64"
    )
    df = df.drop(["end station longitude"], axis=1)

    # bikeid
    df.bikeid = df.bikeid.astype("int16")

    # usertype
    df.usertype = df.usertype.str.lower()
    df.usertype = df.usertype.astype("category")

    return df

cleaned_df_hubway = clean_tripdata(df_hubway)
cleaned_df_hubway.head(1)

/tmp/ipykernel_1303427/1379002695.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.starttime = pd.to_datetime(df.starttime)
/tmp/ipykernel_1303427/1379002695.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.stoptime = pd.to_datetime(df.stoptime)
/tmp/ipykernel_1303427/1379002695.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

,tripduration,starttime,stoptime,bikeid,usertype,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude
0,337,2015-03-01 13:01:50,2015-03-01 13:07:27,769,subscriber,108,harvard university / seas cruft-pierce halls a...,42.377945,-71.116865,88,inman square at vellucci plaza / hampshire st,42.374035,-71.101427


In [95]:
from pathlib import Path 
df = pd.read_parquet(Path("../data/tripdata/2015_01-tripdata.parquet"))
df.dtypes

tripduration                        int64
starttime                  datetime64[ns]
stoptime                   datetime64[ns]
bikeid                              int64
usertype                           object
start_station_id                    int64
start_station_name                 string
start_station_latitude            float64
start_station_longitude           float64
end_station_id                      int64
end_station_name                   string
end_station_latitude              float64
end_station_longitude             float64
dtype: object

### Files named YYYYMM-bluebikes-tripdata.zip

These compressed files contain a csv file and a subdirectory, so they need to be opened with requests, io and zipfile modules.  

In [51]:
import pandas as pd
import requests
import zipfile
import io

url = "https://s3.amazonaws.com/hubway-data/201805-bluebikes-tripdata.zip"
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
df_bluebikes_early = pd.read_csv(z.open("201805-bluebikes-tripdata.csv"))
bluebikes_early_cols = df_bluebikes_early.columns
df_bluebikes_early = df_bluebikes_early.drop(columns=['birth year', 'gender'])
df_bluebikes_early.dtypes

tripduration                 int64
starttime                   object
stoptime                    object
start station id             int64
start station name          object
start station latitude     float64
start station longitude    float64
end station id               int64
end station name            object
end station latitude       float64
end station longitude      float64
bikeid                       int64
usertype                    object
dtype: object

In [52]:
df_bluebikes_early.head(1)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype
0,1177,2018-05-01 00:01:32.4590,2018-05-01 00:21:10.0260,184,Sidney Research Campus/ Erie Street at Waverly,42.357753,-71.103934,189,Kendall T,42.362428,-71.084955,790,Subscriber


In [53]:
import pandas as pd
import requests
import zipfile
import io

url = "https://s3.amazonaws.com/hubway-data/202105-bluebikes-tripdata.zip"
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
df_bluebikes_late = pd.read_csv(z.open("202105-bluebikes-tripdata.csv"))
bluebikes_late_cols = df_bluebikes_late.columns
df_bluebikes_late = df_bluebikes_late.drop(columns=['postal code'])
df_bluebikes_late.dtypes

tripduration                 int64
starttime                   object
stoptime                    object
start station id             int64
start station name          object
start station latitude     float64
start station longitude    float64
end station id               int64
end station name            object
end station latitude       float64
end station longitude      float64
bikeid                       int64
usertype                    object
dtype: object

In [54]:
df_bluebikes_late.head(1)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype
0,609,2021-05-01 00:00:01.0450,2021-05-01 00:10:10.7300,66,Commonwealth Ave at Griggs St,42.349225,-71.132753,400,Lansdowne T Stop,42.347345,-71.100168,4885,Subscriber


In [55]:
set(hubway_cols) & set(bluebikes_early_cols) & set(bluebikes_late_cols)

{'bikeid',
 'end station id',
 'end station latitude',
 'end station longitude',
 'end station name',
 'start station id',
 'start station latitude',
 'start station longitude',
 'start station name',
 'starttime',
 'stoptime',
 'tripduration',
 'usertype'}